### HARI PRASATH S

# Lab 17. Text Classification using CNN-LSTM and Pre-trained Glove Models

In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import nltk
import sklearn
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.optimizers import RMSprop , Adam
from keras.models import Sequential
from keras.layers import *
from nltk.corpus import stopwords 

In [12]:
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.guns",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
    "comp.os.ms-windows.misc",
]
newsgroups = fetch_20newsgroups(categories=categories)
y_true = newsgroups.target

In [33]:
df = pd.DataFrame()
df['text'] = newsgroups.data
df.head()

,text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


In [34]:
df['target'] = [categories[x] for x in y_true]
df

,text,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,rec.motorcycles
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,comp.windows.x
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,comp.windows.x
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,soc.religion.christian
...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,sci.space
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,comp.windows.x
11311,From: westes@netcom.com (Will Estes)\nSubject:...,comp.sys.mac.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,comp.graphics


In [35]:
df.shape

(11314, 2)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11314 entries, 0 to 11313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11314 non-null  object
 1   target  11314 non-null  object
dtypes: object(2)
memory usage: 176.9+ KB


In [37]:
def remove_email_and_user_mentions(text):
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\S*\.edu\S*', '', text)
    return text

In [38]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
df['text'] = df['text'].str.lower()
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
df['text'] = df['text'].apply(remove_email_and_user_mentions)
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
df

,text,target
0,lerxstwamumdedu wheres thing subject car nntpp...,rec.motorcycles
1,guykuocarsonuwashingtonedu guy kuo subject si ...,comp.windows.x
2,twillisececnpurdueedu thomas e willis subject ...,comp.windows.x
3,jgreenamber joe green subject weitek p9000 org...,comp.graphics
4,jcmheadcfaharvardedu jonathan mcdowell subject...,soc.religion.christian
...,...,...
11309,jimzisfeinfactorycom jim zisfein subject migra...,sci.space
11310,ebodinpearltuftsedu subject screen death mac p...,comp.windows.x
11311,westesnetcomcom estes subject mounting cpu coo...,comp.sys.mac.hardware
11312,stevehcrlgw steven collins subject sphere 4 po...,comp.graphics


In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
max_sequence_length = 100 
X = pad_sequences(X, maxlen=max_sequence_length, padding='post', truncating='post')

In [49]:
class_labels = df['target'].unique()
class_mapping = {label: index for index, label in enumerate(class_labels)}
df['target'] = df['target'].map(class_mapping)

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['target'], test_size=0.3, random_state=42)

In [54]:
maxlen = 100  
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')


In [58]:
model = Sequential()
embedding_dim = 128  
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
lstm_units = 64  
model.add(LSTM(units=lstm_units))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(20, activation='softmax'))

In [59]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the results
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss:.4f}')

Epoch 1/20
223/223 - 49s - loss: 2.9133 - accuracy: 0.0700 - val_loss: 2.8616 - val_accuracy: 0.0770 - 49s/epoch - 218ms/step
Epoch 2/20
223/223 - 43s - loss: 2.7020 - accuracy: 0.1054 - val_loss: 2.4969 - val_accuracy: 0.1351 - 43s/epoch - 195ms/step
Epoch 3/20
223/223 - 44s - loss: 2.3412 - accuracy: 0.1736 - val_loss: 2.2107 - val_accuracy: 0.2235 - 44s/epoch - 198ms/step
Epoch 4/20
223/223 - 44s - loss: 2.0826 - accuracy: 0.2496 - val_loss: 2.0763 - val_accuracy: 0.2197 - 44s/epoch - 198ms/step
Epoch 5/20
223/223 - 43s - loss: 1.8133 - accuracy: 0.3426 - val_loss: 2.3332 - val_accuracy: 0.2235 - 43s/epoch - 195ms/step
Epoch 6/20
223/223 - 43s - loss: 1.4176 - accuracy: 0.5033 - val_loss: 1.7308 - val_accuracy: 0.4280 - 43s/epoch - 194ms/step
Epoch 7/20
223/223 - 43s - loss: 0.9868 - accuracy: 0.6753 - val_loss: 1.6513 - val_accuracy: 0.4747 - 43s/epoch - 194ms/step
Epoch 8/20
223/223 - 43s - loss: 0.6969 - accuracy: 0.7836 - val_loss: 1.5023 - val_accuracy: 0.5253 - 43s/epoch - 194

In [60]:
model1 = Sequential()
embedding_dim = 128  
model1.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model1.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
lstm_units = 64  
model1.add(LSTM(units=lstm_units))
model1.add(Dense(units=64, activation='relu'))
model1.add(Dense(20, activation='softmax'))

In [62]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history1 = model1.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)
loss, accuracy = model1.evaluate(X_test, y_test, verbose=0)

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Loss: {loss:.4f}')

Epoch 1/20
223/223 - 43s - loss: 2.6757 - accuracy: 0.1019 - val_loss: 2.4159 - val_accuracy: 0.1174 - 43s/epoch - 191ms/step
Epoch 2/20
223/223 - 40s - loss: 2.0959 - accuracy: 0.1894 - val_loss: 2.0468 - val_accuracy: 0.2222 - 40s/epoch - 179ms/step
Epoch 3/20
223/223 - 38s - loss: 1.6946 - accuracy: 0.3099 - val_loss: 1.7098 - val_accuracy: 0.3220 - 38s/epoch - 171ms/step
Epoch 4/20
223/223 - 36s - loss: 1.2826 - accuracy: 0.4578 - val_loss: 1.5062 - val_accuracy: 0.4356 - 36s/epoch - 163ms/step
Epoch 5/20
223/223 - 37s - loss: 0.9465 - accuracy: 0.6105 - val_loss: 1.2251 - val_accuracy: 0.5758 - 37s/epoch - 168ms/step
Epoch 6/20
223/223 - 41s - loss: 0.6835 - accuracy: 0.7376 - val_loss: 1.2895 - val_accuracy: 0.6338 - 41s/epoch - 182ms/step
Epoch 7/20
223/223 - 38s - loss: 0.4291 - accuracy: 0.8507 - val_loss: 1.1438 - val_accuracy: 0.6869 - 38s/epoch - 172ms/step
Epoch 8/20
223/223 - 39s - loss: 0.2938 - accuracy: 0.9105 - val_loss: 1.4224 - val_accuracy: 0.6717 - 39s/epoch - 176

In [64]:
pip install gensim


  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ab/b0/d58dc405fd60ab546ca714321235dc2d455b2dc06bfb4fc1092940c749fc/gensim-4.3.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB 435.7 kB/s eta 0:00:56
   ---------------------------------------- 0.0/24.0 MB 245.8 kB/s eta 0:01:38
   ---------------------------------------- 0.1/24.0 MB 297.7 kB/s eta 0:01:21
   ---------------------------------------- 0.1/24.0 MB 374.1 kB/s eta 0:01:04
   ---------------------------------------- 0.1/24.0 MB 425.3 kB/s eta 0:00:57
   ---------------------------------------- 0.

In [65]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [67]:
glove_file = "glove.6B.100d.txt"
glove_word2vec_file = "glove.6B.100d.txt.word2vec"
glove2word2vec(glove_file, glove_word2vec_file)
glove_embeddings = KeyedVectors.load_word2vec_format(glove_word2vec_file, binary=False)

C:\Users\HP\AppData\Local\Temp\ipykernel_9292\889183730.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, glove_word2vec_file)


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
word_index = tokenizer.word_index

# Create an embedding matrix
embedding_dim = 100  # GloVe vectors are 100-dimensional
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

In [ ]:
model2 = Sequential()
model2.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model2.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
lstm_units = 64  
model2.add(LSTM(units=lstm_units))
model2.add(Dense(units=64, activation='relu'))
model2.add(Dense(20, activation='softmax'))

In [ ]:
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

In [ ]:
model1 = Sequential() 
model1.add(Embedding(vocab_size, embedding_dim)) 
model1.add(Conv1D(filters=32, kernel_size=5, strides=1, activation='relu'))
model1.add(MaxPooling1D((2))) 
model1.add(LSTM(embedding_dim)) 
model1.add(Dense(128, activation= 'relu'))
model1.add(Dense(6, activation='softmax')) 
model1.summary()

In [27]:
model2 = Sequential()
model2.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model2.add(Conv1D(filters=32, kernel_size=5, strides=1, activation='relu'))
model2.add(MaxPooling1D((2))) 
model2.add(LSTM(embedding_dim)) 
model2.add(Dense(128, activation= 'relu'))
model2.add(Dense(6, activation='softmax')) 
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 300)          1500000   
                                                                 
 conv1d_1 (Conv1D)           (None, 196, 32)           48032     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 98, 32)            0         
 g1D)                                                            
                                                                 
 lstm_2 (LSTM)               (None, 50)                16600     
                                                                 
 dense_4 (Dense)             (None, 128)               6528      
                                                                 
 dense_5 (Dense)             (None, 6)                 774       
                                                      

In [30]:
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(x_train,y_train, epochs=5, verbose=2, validation_split=0.2)

Epoch 1/5
45/45 - 12s - loss: 1.6754 - accuracy: 0.2338 - val_loss: 1.6813 - val_accuracy: 0.2022 - 12s/epoch - 263ms/step
Epoch 2/5
45/45 - 5s - loss: 1.6716 - accuracy: 0.2338 - val_loss: 1.6780 - val_accuracy: 0.2022 - 5s/epoch - 116ms/step
Epoch 3/5
45/45 - 5s - loss: 1.6682 - accuracy: 0.2338 - val_loss: 1.6749 - val_accuracy: 0.2022 - 5s/epoch - 116ms/step
Epoch 4/5
45/45 - 5s - loss: 1.6650 - accuracy: 0.2338 - val_loss: 1.6721 - val_accuracy: 0.2022 - 5s/epoch - 115ms/step
Epoch 5/5
45/45 - 5s - loss: 1.6621 - accuracy: 0.2338 - val_loss: 1.6691 - val_accuracy: 0.2022 - 5s/epoch - 122ms/step
